# Imports

In [12]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()
from os.path import join as path_join

from config import DATA_DIR

# Load data

In [15]:
data = pd.read_parquet(path_join(DATA_DIR, 'movie_reviews.parquet'))
print(data.shape)
data.sample(5, random_state=124)

(2000, 2)


,text,target
1769,men in black is an explosive mix of science fi...,1
1473,"a sci fi / comedy starring jack nicholson , pi...",1
1840,the question isn ' t why has grease been reiss...,1
1537,before even seeing a single frame of the film ...,1
1866,""" through a spyglass , i could see everything ...",1


In [5]:
data.target.value_counts()

1    1000
0    1000
Name: target, dtype: int64

# If all the revies are on english?

In [12]:
from langdetect import detect_langs as lang_detector
from langdetect.lang_detect_exception import LangDetectException

In [13]:
def detect_langs(text: str, default='en', top_rated: int=3, threshold: float=0.1) -> list:
    """
    Function returns 3 the most probable languages the text is written on.
    Probability of each languages have to beat threshold: 0.5 to be put in the returned list
    :param text: text to detect language it was written on
    :param default: default language to return if error was caught
    :param top_rated: how many languages to return
    :param threshold: threshold to filter the languages with smaller probability.
                      Have to be between values: [0.1, 1.0)
                      PS: If threshold is bigger than 0.5 it is logically that the function returns a list of one item.
    :return: list of languages. first is the most possible
    """
    try:
        preds = lang_detector(text.lower())
        if preds == 'unknown':
            return [default]
        preds = sorted(filter(lambda x: x.prob >= threshold, preds), reverse=True, key=lambda x: x.prob)[:top_rated]
        return [pred.lang for pred in preds]
    except LangDetectException:
        logging.debug('LangDetectException', exc_info=1)
        return [default]

In [18]:
langs = data.text.progress_apply(lambda x: detect_langs(x, default='unknown', top_rated=1)[0])
is_english = all(map(lambda x: x == 'en', langs))
print(f'Is all text on english language: {is_english}')
print(pd.Series(langs).value_counts())

Is all text on english language: True
en    2000
Name: text, dtype: int64


# Set of symbols

In [22]:
import re
from itertools import chain

In [28]:
symbols = set(chain(*data.text))
chars  = set()
spec   = set()
digits = set()
for sym in symbols:
    if sym.isalpha():
        chars.update(sym)
    elif sym.isdigit():
        digits.update(sym)
    else:
        spec.update(sym)
print(f'Chars  : {sorted(chars)}')
print(f'Digits : {sorted(digits)}')
print(f'Spec   : {sorted(spec)}')

Chars  : ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Digits : ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Spec   : ['\x05', '\x12', '\x13', '\x14', '\x16', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


# What phrases are there

In [29]:
import re

In [56]:
print(re.search(r"(https?\://)\S+", plain_text))
print(re.search(r"(www\://)\S+", plain_text))
print(re.search(r"@[^:| ]+:? ?", plain_text))
print(re.search(r"#+ ?", plain_text))
print(re.search(r'[-+]?\d+(\.[0-9]*)?', plain_text))

None
None
<re.Match object; span=(645863, 645868), match='@$&% '>
<re.Match object; span=(60007, 60009), match='# '>
<re.Match object; span=(2309, 2311), match='20'>


# Create positive/negative

In [25]:
import os
import re
from config import DATA_DIR

from typing import Tuple, List

In [19]:
def get_dictionaries() -> Tuple[List[str], List[str]]:
    with open(os.path.join(DATA_DIR, 'word_dictionaries/negative.txt'), 'r') as file:
        negative = [s.strip().lower() for s in file.readlines()]
    with open(os.path.join(DATA_DIR, 'word_dictionaries/positive.txt'), 'r') as file:
        positive = [s.strip().lower() for s in file.readlines()]
    return negative, positive


NEGATIVE, POSITIVE = get_dictionaries()
plain_text = ' '.join(data['text'])

In [20]:
def update_dictionary(dictionary, text):
    new_dict = {word for word in dictionary if word in text}
    len_new_dict = len(new_dict)
    print(f'New dict include {len_new_dict} words ({len_new_dict / len(dictionary)*100:.2f}%)')
    return new_dict

NEGATIVE = update_dictionary(NEGATIVE, plain_text)
POSITIVE = update_dictionary(POSITIVE, plain_text)

New dict include 3387 words (70.81%)
New dict include 1527 words (76.12%)


In [24]:
# Count of negative positive words
d = {f'not {word}' for word in POSITIVE if f'not {word}' in plain_text}
print(f'Negative postive words: {len(d)} ({len(d) / len(POSITIVE)*100:.2f}%)')

Negative postive words: 140 (9.17%)


In [31]:
NEGATIVE.update(d)

In [37]:
# Do not like pattern
len(re.findall(r"(do not)|(don't)|(does not)|(doesn't)|(didn't)|(did not)|(has not)|(hasn't)|(have not)|(haven't) like", plain_text))

711

In [33]:
def write_dictionaries(positive, negative):
    with open(os.path.join(DATA_DIR, 'word_dictionaries/updated_negative.txt'), 'w') as file:
        file.write('\n'.join(negative))
    with open(os.path.join(DATA_DIR, 'word_dictionaries/updated_positive.txt'), 'w') as file:
        file.write('\n'.join(positive))
write_dictionaries(positive=POSITIVE, negative=NEGATIVE)